In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

In [7]:
# --- 1. Khai báo thông tin ---
BASE_URL = "https://homedy.com/ban-nha-dat/p{}"  # Template URL với số trang
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [8]:
# --- 2. Khởi tạo danh sách dữ liệu ---
data = []
max_data = 2000  # Số lượng mẫu tối đa muốn thu thập
page = 1

In [9]:
# --- 3. Vòng lặp qua các trang cho đến khi đủ dữ liệu ---
while len(data) < max_data:
    # Tạo URL cho trang hiện tại
    if page == 1:
        url = "https://homedy.com/ban-nha-dat"
    else:
        url = BASE_URL.format(page)
    
    print(f"Đang crawl trang {page}: {url}")
    
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        
        # Kiểm tra nếu trang không tồn tại (redirect hoặc lỗi 404)
        if response.url != url and "ban-nha-dat" not in response.url:
            print("Đã đến trang cuối cùng. Dừng crawl.")
            break
            
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi gửi yêu cầu đến trang {page}: {e}")
        break

    soup = BeautifulSoup(response.content, 'html.parser')
    list_items = soup.find_all('div', class_='product-content')
    
    # Nếu không tìm thấy tin đăng nào nữa thì dừng
    if not list_items:
        print("Không tìm thấy tin đăng nào. Kết thúc crawl.")
        break

Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://homedy.com/ban-nha-dat
Đang crawl trang 1: https://hom

KeyboardInterrupt: 

In [11]:
    # --- 4. Trích xuất dữ liệu từ trang hiện tại ---
    items_count = 0
    for item in list_items:
        try:
            # Trích xuất giá
            price_element = item.find('span', class_='price')
            price = price_element.get_text(strip=True) if price_element else "Không rõ"
            
            # Trích xuất diện tích
            acreage_element = item.find('span', class_='acreage')
            acreage = acreage_element.get_text(strip=True) if acreage_element else "Không rõ"
            
            # Trích xuất địa chỉ
            address_element = item.find('li', class_='address')
            address = address_element.get_text(strip=True) if address_element else "Không rõ"
            
            # Chỉ thêm dữ liệu nếu có đủ thông tin cơ bản
            if price != "Không rõ" or acreage != "Không rõ" or address != "Không rõ":
                data.append({
                    'Địa chỉ': address,
                    'Diện tích': acreage,
                    'Giá': price,
                    'Trang': page
                })
                items_count += 1
                
            # Dừng nếu đã đủ 2000 mẫu
            if len(data) >= max_data:
                break
                
        except Exception as e:
            print(f"Lỗi khi trích xuất dữ liệu từ một item: {e}")
            continue
    
    print(f"Đã thu thập {items_count} mẫu từ trang {page}. Tổng cộng: {len(data)}/{max_data}")
    
    # Tăng số trang
    page += 1
    
    # Tạm dừng ngẫu nhiên từ 2-5 giây giữa các request để tránh bị chặn
    sleep_time = random.uniform(2, 5)
    print(f"Tạm dừng {sleep_time:.2f} giây...")
    time.sleep(sleep_time)

Đã thu thập 18 mẫu từ trang 2. Tổng cộng: 36/2000
Tạm dừng 3.64 giây...


In [ ]:
# --- 5. Tạo DataFrame và lưu file ---
if data:
    df = pd.DataFrame(data)
    print(f"\nĐã thu thập thành công {len(df)} mẫu dữ liệu!")
    print("\n5 mẫu đầu tiên:")
    print(df.head())
    
    # Lưu file với timestamp để tránh ghi đè
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f'vietnam_housing_data_{timestamp}.csv'
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\nĐã lưu dữ liệu vào file '{filename}'")
else:
    print("Không tìm thấy dữ liệu nào để lưu!")